# TP3 - Parte 2: Baseline

Vamos a construir un modelo muy sencillo para saber qué es lo peor que podemos hacer, en general esta es una tarea muy importante que queremos que repitan en sus proyectos de machine learning. ¿Por qué?

- Navaja de Ockham: “Cuando se ofrecen dos o más explicaciones de un fenómeno, es preferible la explicación completa más simple; es decir, no deben multiplicarse las entidades sin necesidad.” ¿Para qué desarrollar un modelo super complejo si capaz es peor o casi igual que uno muy sencillo?
- Nos sirve para saber si estamos usando bien los modelos más complejos, si su score nos da peor al baseline probablemente se deba a un error de código.
- Nos sirve para rápidamente saber que tan complejo es un problema.
- Los modelos simples son fáciles de entender.

Se deben crear al menos dos features numéricas y dos features categóricas para entrenar una regresión logística, utilizando búsqueda de hiperparametros, realizando los encodings correspondientes y garantizando la reproducibilidad de los resultados cuando el notebook corriera varias veces. A su vez, usar un embedding para el campo text.

Conteste las preguntas:

- ¿Cuál es el mejor score de validación obtenido? (¿Cómo conviene obtener el dataset para validar?)
- Al predecir con este modelo para la competencia, ¿Cúal es el score obtenido? (guardar el csv con predicciones para entregarlo después)
- ¿Qué features son los más importantes para predecir con el mejor modelo? Graficar.

## Imports y carga de datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("../data/processed/train_modificado.csv", index_col=0)

In [ ]:
df.head(5)

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
X.head(5)

In [ ]:
y.head(5)

## División en Train y Validation

Usando el parámetro stratify me aseguro que la proporción entre targets positivos y negativos se mantenga luego del split, igual voy a querer comprobarlo:

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)

In [ ]:
print("Ratio original:", df['target'].value_counts(normalize=True))
print("Ratio y_train:", y_train.value_counts(normalize=True))
print("Ratio y_validation:", y_validation.value_counts(normalize=True))

Podemos ver que las proporciones se mantuvieron casi iguales por lo que nos aseguramos no confundir al modelo en este aspecto.

In [ ]:
X_train.head(5)

In [ ]:
y_train.head(5)

## Escalado de los datos y embedding TF-IDF

Ahora que ya tenemos la separación en train y validation, podemos procesar train sin leakear los targets de validation lo que nos permite sacar features que no dependan unicamente de las otras features de la misma fila, sino también utilizando estadísticas de todo el set de train.

Esto nos permitirá hacer un embedding TF-IDF sin leakear y_train y también escalar las features numéricas para que la regresión logística converga mejor.

## Escalado de las features numéricas

Las features numéricas que tenemos son:
- location_count
- tweet_length
- words_count
- num_uppercase_letters
- num_uppercase_words
- num_special_chars
- num_digits

Asique solo vamos a querer escalar estas:


In [ ]:
cols_numericas = [
    "tweet_length",
    "words_count",
    "num_uppercase_letters",
    "num_uppercase_words",
    "num_special_chars",
    "num_digits"
]

In [ ]:
scaler = StandardScaler()

Para escalar, necesito quedarme solo con las columnas numéricas y hacer el fit solo con train

In [ ]:
scaler.fit(X_train[cols_numericas])

Ahora si, podemos hacer el transform y sobreescribir las columnas por los valores escalados:

In [ ]:
X_train_scaled = X_train.copy()
X_validation_scaled = X_validation.copy()

X_train_scaled[cols_numericas] = scaler.transform(X_train[cols_numericas])
X_validation_scaled[cols_numericas] = scaler.transform(X_validation[cols_numericas])

In [ ]:
X_train_scaled.head(2)

In [ ]:
X_validation_scaled.head(2)

## Embedding TF-IDF

### Eliminación de Stopwords

Para realizar este primer embedding que vamos a utilizar para nuestra regresión logística, ya podemos empezar a procesar el texto. El TfidfVectorizer de sklearn nos ahorra tener que hacer un tokenizado manual, pero puede verse afectado por la presencia de stopwords

Por esto, vamos a eliminar primero las `stopwords` antes de aplicarlo. `TfidfVectorizer` tiene una opción para eliminar `stopwords` en su procesado, pero su [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#:~:text=given%20callable%20analyzer.-,stop_words,-%7B%E2%80%98english%E2%80%99%7D%2C%20list%2C%20default) indica que su uso tiene algunos inconvenientes:

> There are several known issues with ‘english’ and you should consider an alternative.

Por lo tanto, vamos a utilizar la lista de `stopwords` que `nltk.corpus` para el idioma inglés. Nuestro trabajo previo con `langdetect` nos hizo observar que podemos asumir todos los tweets como entradas en ese idioma y por lo tanto no debemos preocuparnos por las `stopwords` de otros idiomas.

El lado positivo es que corpus nos da una lista de palabras, y luego podemos pasar esa lista al parámetro de `TfidfVectorizer`:

> If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens. Only applies if `analyzer == 'word'`.

Esto funciona ya que el valor default de `analyzer` es efectivamente `'word'`. Además, no es case-sensitive ya que utilizaremos el parámetro `lowercase=True`.

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwordsEng = list(stopwords.words('english'))
stopwordsEng[0:15]

OBSERVACIÓN: Cómo no estamos tomando stopwords personalizadas según la frecuencia en nuestro dataset de train ni nada similar. Podríamos hacer este filtro directamente en el dataframe original.

OBSERVACIÓN: Podriamos pensar que deberíamos haber computado las features anteriores con los tweets sin stopwords, y puede ser cierto. Aunque a su vez las features antes de eliminar las stopwords pueden tener información adicional relacionadas con la forma de escribir. Por falta de tiempo no haré un análisis comparando las dos posibilidades.

### Vectorizado del Texto

Para el vectorizado vamos a utilizar TweetTokenizer como tokenizador, que es un tokenizador especializado en tweets.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import TweetTokenizer

In [ ]:
tokenizer = TweetTokenizer()

Vamos a hacer un CountVectorizer previo con este tokenizador primero, simplemente porque quiero estimar cuál es la cantidad de features/palabras distintas en X_train.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer(
    lowercase = True,
    tokenizer=tokenizer.tokenize,
    token_pattern = None,
    stop_words = stopwordsEng
)
v.fit(X_train["text"])
len(v.vocabulary_)

Tenemos Casi 20k tokens distintos, por lo tanto, voy a probar con algunos valores distintos de max_features a ver cuál me da el mejor resultado:

In [ ]:
MAX_FEATURES = [
    2000, 4000, 6000, 8000, 10000, 12000
]

Para iniciar, probamos solo con el primer valor y mostramos los pasos:

In [ ]:
vectorizer = TfidfVectorizer(
    lowercase = True,
    max_features=MAX_FEATURES[0],
    tokenizer=tokenizer.tokenize,
    token_pattern = None,
    stop_words = stopwordsEng
)

In [ ]:
X_train_tfidf = vectorizer.fit_transform(X_train['text'])

In [ ]:
feature_names = vectorizer.get_feature_names_out()

In [ ]:
X_train_tfidf.toarray()[0:10]

In [ ]:
X_train_tfidf_dense = pd.DataFrame(X_train_tfidf.toarray(), columns=feature_names, index=X_train.index)

In [ ]:
X_train_tfidf_dense.head()

In [ ]:
pd.concat([X_train, X_train_tfidf_dense], axis=1)

Ahora juntamos todo esto en una función para poder aplicarlo automáticamente tanto a train como validation con los parámetros a elección:

In [ ]:
def vectorizadoTfidf(X_train: pd.DataFrame, X_validation: pd.DataFrame, features: int, columna_texto: str, tokenizer_func=None):
    X_train_copy = X_train.copy()
    X_validation_copy = X_validation.copy()

    vectorizer = TfidfVectorizer(
        lowercase = True,
        max_features=features,
        tokenizer=tokenizer_func,
        token_pattern = None,
        stop_words = stopwordsEng
    )
    vectorizer.fit(X_train_copy[columna_texto])
    feature_names = vectorizer.get_feature_names_out()

    # train
    train_transformed = vectorizer.transform(X_train_copy[columna_texto])
    train_transformed_dense = pd.DataFrame(train_transformed.toarray(), columns=feature_names, index=X_train_copy.index)
    train_full = pd.concat([X_train_copy, train_transformed_dense], axis=1)

    # validation
    validation_transformed = vectorizer.transform(X_validation_copy[columna_texto])
    validation_transformed_dense = pd.DataFrame(validation_transformed.toarray(), columns=feature_names, index=X_validation_copy.index)
    validation_full = pd.concat([X_validation_copy, validation_transformed_dense], axis=1)

    return train_full, validation_full, vectorizer # Devuelvo el vectorizer por si quiero reutilizarlo

Ahora, podemos aplicarle el vectorizado a `X_train_scaled` que es donde ya escalamos, luego dropeamos las columnas que no nos sirven para el modelo (keyword, location y text) y podemos aplicar la regresión lineal.

Podemos hacer esto en un loop para iterar sobre la cantidad de features en el vectorizado y comparar resultados:

In [ ]:
def tokenizar_texto_tweet(text: str):
  return tokenizer.tokenize(text)

In [ ]:
X_train_vectorizado, X_validation_vectorizado, vectorizer = vectorizadoTfidf(X_train_scaled, X_validation_scaled, 2000, "text", tokenizar_texto_tweet)

In [ ]:
X_train_vectorizado.head(1)

Finalmente dropeamos las columnas que no queremos y ya podemos aplicar la regresión lineal:

In [ ]:
X_train_final = X_train_vectorizado.drop(columns=['keyword','location','text'])
X_validation_final = X_validation_vectorizado.drop(columns=['keyword','location','text'])

## Regresión Lineal

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import loguniform
from sklearn.metrics import f1_score

In [ ]:
lr = LogisticRegression(max_iter=250)
# No uso max_iter como hiperparámetros porque no afecta al output, sino que limita cuantas iteraciones se realizan hasta convergencia.
# El default es 100 asique lo seteo en 250 para seguridad

Para la búsqueda de hiperparámetros, vamos a hacer un RandomSearch primero para acotar el espacio de posibles valores y acercarnos a algunas "zonas" mejores que otras. Y luego vamos a usar GridSearch con un set más reducido para quedarnos con lo mejor.

Como LogisticRegression no tiene demasiados parámetros de espacio continuo (sino que la mayoría son opciones y espacios discretos), usaremos los que dieron mejor perfomance pero aprovecharemos para aplicar un GridSearch sobre max_iter. Este approach no aporta mucho más que un GridSearch y listo, pero lo vamos a hacer así ya que probablemente luego lo haremos igual con otros modelos.

In [ ]:
params = {
    'C': loguniform(1e-3,100),
    'penalty': ['l2', 'l1'],
    'solver': ['lbfgs','saga'],
    'class_weight': [None,'balanced']
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=lr,
    param_distributions=params,
    n_iter=150,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    random_state=123
)

In [ ]:
random_search.fit(X_train_final, y_train)

In [ ]:
print("Mejores hiperparámetros del RandomSearch:")
print(random_search.best_params_)
print("Mejor F1 obtenido (CV):", random_search.best_score_)

In [ ]:
best_random_params = random_search.best_params_

In [ ]:
C_best = best_random_params["C"]

In [ ]:
grid_params = {
    "C": [C_best *0.25, C_best * 0.5, C_best * 0.75, C_best, C_best * 1.25, C_best * 1.5, C_best * 1.75, C_best * 2],
    "penalty": [best_random_params["penalty"]],
    "solver": [best_random_params["solver"]],
    "class_weight": [best_random_params["class_weight"]]
}

In [ ]:
lr_grid = LogisticRegression(max_iter=250)

In [ ]:
grid_search = GridSearchCV(
    estimator=lr_grid,
    param_grid=grid_params,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train_final, y_train)

In [ ]:
print("\nMejores hiperparámetros finales (GridSearch):")
print(grid_search.best_params_)
print("Mejor F1 obtenido (CV):", grid_search.best_score_)

- Surge la duda: Por qué el Mejor F1 obtenido de RandomSearch puede ser mejor que el Mejor F1 obtenido de GridSearch? Investigando encontré que como utilizamos Cross-Validation (cv=3), puede haber variaciones entre los folds usados por RandomSearch y GridSearch, que obtengan resultados distintos. Por eso hay esa pequeña variación.

Nos quedamos entonces con el mejor estimador con los hiperparámetros buscados y usamos ese. Vamos a compararlo a su vez con la instancia por defecto:

In [ ]:
lr_final = grid_search.best_estimator_

In [ ]:
lr.fit(X_train_final, y_train)

In [ ]:
f1 = f1_score(y_train, lr.predict(X_train_final))
print("F1-score en train de lr default:", f1)
f1 = f1_score(y_train, lr_final.predict(X_train_final))
print("F1-score en train de lr con hiperparámetros ajustados:", f1)

In [ ]:
lr_final.get_params()

In [ ]:
lr.get_params()

Ahora podemos predecir sobre validación y medir el score con F1:

In [ ]:
f1 = f1_score(y_validation, lr.predict(X_validation_final))
print("F1-score en validation de lr default:", f1)
f1 = f1_score(y_validation, lr_final.predict(X_validation_final))
print("F1-score en validation de lr con hiperparámetros ajustados:", f1)

Tenemos un puntaje bastante potable. Pero vamos a barrer algunos thresholds hasta encontrar el umbral de corte para la regresión logistica que nos maximice el F1-score en validation.

In [ ]:
probabilidades = lr_final.predict_proba(X_validation_final)[:,1]

In [ ]:
thresholds = np.linspace(0, 1, 200)
f1_scores = []

for thr in thresholds:
    y_pred_thr = (probabilidades >= thr).astype(int)
    f1_scores.append(f1_score(y_validation, y_pred_thr))

best_thr = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
best_y_pred = (probabilidades >= best_thr).astype(int)

In [ ]:
print("Mejor threshold:", best_thr)
print("Mejor F1:", best_f1)

La verdad que un score así es mejor de lo esperado. Además, incluso habiedo ajustado los hiperparámetros, todavía podriamos iterar sobre distintos valores de MAX_FEATURES para ver cuál nos da un mejor resultado. Sin embargo por falta de tiempo voy a quedarme con este resultado para la Regresión Logística.

## Respondiendo Preguntas

Pasamos ahora a contestar las preguntas de la consigna.

1) ¿Cuál es el mejor score de validación obtenido? (¿Cómo conviene obtener el dataset para validar?)

El mejor score obtenido fue 0.747 o casi 0.75. Para obtener el dataset para validar, lo que conviene es hacer un split que respete las distribuciones de targets positivos y negativos. Como no hay features temporales no debemos preocuparnos del time-travel. Además, hicimos el escalado y embedding de los features luego del split, lo que previene que haya data leaks respecto al target.

Un mejor approach podría ser utilizar K-fold cross-validation para buscar que el modelo se comporte mejor ante nuevos features, pero no lo hice.


2) Al predecir con este modelo para la competencia, ¿Cúal es el score obtenido? (guardar el csv con predicciones para entregarlo después)

Para predecir para la competencia, vamos a cargar test_modificado.csv en memoria, aplicar el escalado y embedding y luego realizar las predicciones con el modelo que obtuvo el mejor puntaje F1.

Luego vamos a guardar las predicciones obtenidas con el mejor puntaje de validación en un .csv y luego vamos a subirlo a kaggle para ver el puntaje obtenido.

In [ ]:
X_test = pd.read_csv("../data/processed/test_modificado.csv", index_col=0)

In [ ]:
X_test_scaled = X_test.copy()
X_test_scaled[cols_numericas] = scaler.transform(X_test[cols_numericas])

In [ ]:
X_test_scaled.head(2)

Reutilizamos el vectorizer:

In [ ]:
test_transformed = vectorizer.transform(X_test_scaled['text'])
test_transformed_dense = pd.DataFrame(test_transformed.toarray(), columns=feature_names, index=X_test_scaled.index)
test_full = pd.concat([X_test_scaled, test_transformed_dense], axis=1)

In [ ]:
X_test_final = test_full.drop(columns=['keyword','location','text'])
X_test_final.head(2)

In [ ]:
probabilidades = lr_final.predict_proba(X_test_final)[:,1]

In [ ]:
test_pred = (probabilidades >= best_thr).astype(int)

In [ ]:
submit = pd.DataFrame(test_pred, columns=['target'], index = X_test_final.index)

In [ ]:
submit.head()

In [ ]:
submit.info()

In [ ]:
submit.to_csv("../data/processed/submit_baseline.csv", index=True)

Ahora subimos el submit a Kaggle y vemos el puntaje obtenido fue:
> **Score: 0.78884**



3) ¿Qué features son los más importantes para predecir con el mejor modelo? Graficar.

Para un modelo de regresión logística, las importancias de las features provienen directamente de los coeficientes que se le asigna a cada una de ellas. Asique podemos obtenerlos, ordenarlos según su valor absoluto y graficarlos:

In [ ]:
coef = lr.coef_[0]

In [ ]:
feature_names = X_train_final.columns

In [ ]:
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coef': coef,
    'abs_coef': np.abs(coef)
}).sort_values('abs_coef', ascending=False)

Podemos ver las 20 features más importantes:

In [ ]:
importance_df.head(20)

Y podemos graficar las top_n features más importantes:

In [ ]:
top_n = 20
top_features = importance_df.head(top_n)

colors = np.where(top_features['coef'] >= 0, "seagreen", "darkred")

plt.figure(figsize=(8, 10))
plt.barh(top_features['feature'], top_features['abs_coef'], color=colors)
plt.gca().invert_yaxis()
plt.xlabel("Importancia (|coef|)")
plt.title(f"Top {top_n} features más importantes según Logistic Regression")
plt.text(0.95, 0.05, "Colores:\nCoef > 0 → seagreen\nCoef < 0 → darkred",
         transform=plt.gca().transAxes,
         fontsize=10,
         ha='right',
         bbox=dict(facecolor='white', alpha=0.7))
plt.tight_layout()
plt.show()

Es interesante ver que, aún no ponderando las keywords. Muchisimas de las features más importantes provienen del embedding hecho sobre el texto y que muchas de ellas son palabras que probablemente sean keywords.

Más interesante aún, coloreando las barras según el signo del coeficiente, podemos ver que hay features/tokens del embedding que disminuyen el target haciendo menos probable que el tweet sea de una catástrofe. Tokens como 'love' tiene mucho sentido que tomen coeficientes negativos.

## Persistencia de Datos

El procesado extra de los datos no demoró mucho tiempo, y teniendo el código decidí hacer un nuevo collab de feature engineering para hacer un preprocesado de datos en limpio mucho mejor y completo. Así tendremos mejores features para los modelos más avanzados.